In [1]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.
/Users/yaqiongzhang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
img_width, img_height = 150, 150
epochs = 50
batch_size = 16
split = 0.9

In [3]:
X_train=np.load('X_train.npy')
target_train=np.load('target_train.npy')
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train[:1604], train_size=split)
nb_train_samples = len(X_train_cv)
nb_validation_samples = len(X_valid)

In [7]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    bottleneck_features_train = model.predict(X_train_cv)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)

    bottleneck_features_validation = model.predict(X_valid)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)

In [8]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')

    validation_data = np.load('bottleneck_features_validation.npy')
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    model.fit(train_data, y_train_cv,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, y_valid))
    
    model.save_weights('top_model_weights_path')


In [9]:
save_bottlebeck_features()
train_top_model()

Train on 1443 samples, validate on 161 samples
Epoch 1/50
1443/1443 [==============================] - 1s 686us/step - loss: 0.6325 - acc: 0.6979 - val_loss: 0.4719 - val_acc: 0.7826
Epoch 2/50
1443/1443 [==============================] - 1s 429us/step - loss: 0.4477 - acc: 0.7762 - val_loss: 0.5116 - val_acc: 0.6894
Epoch 3/50
1443/1443 [==============================] - 1s 421us/step - loss: 0.4032 - acc: 0.8053 - val_loss: 0.4153 - val_acc: 0.8012
Epoch 4/50
1443/1443 [==============================] - 1s 587us/step - loss: 0.3730 - acc: 0.8219 - val_loss: 0.4755 - val_acc: 0.7329
Epoch 5/50
1443/1443 [==============================] - 1s 519us/step - loss: 0.3537 - acc: 0.8288 - val_loss: 0.3498 - val_acc: 0.8012
Epoch 6/50
1443/1443 [==============================] - 1s 443us/step - loss: 0.3466 - acc: 0.8399 - val_loss: 0.3806 - val_acc: 0.8137
Epoch 7/50
1443/1443 [==============================] - 1s 491us/step - loss: 0.3315 - acc: 0.8489 - val_loss: 0.3636 - val_acc: 0.8447
E

NameError: name 'top_model_weights_path' is not defined